In [1]:
import datetime
import geopandas as gp
from shapely.geometry import Point
import pandas as pd

In [2]:

class Bower:
    """developed in:
    Bowers, K.J. et al. 2004. Prospective Hot-SpottingThe Future of Crime Mapping?
    The British journal of criminology. 44, 5 (Sep. 2004), 641–658.
    """
    def __str__(self):
        return (
            'Bower method: weighted by distance and time,'
            'bandwidth={}, time window={}, verbose={}'.format(
                self.bw, self.tw, self.verbose))

    def __init__(self, bw=400, tw=60, verbose=0):
        """

        :param bw: bandwidth, int, default = 400
        :param tw: time window, int, default = 60, number of days in the past to be considered
        :param verbose: level of verbosity
        """
        self.bw=bw
        self.tw=tw
        self.verbose=verbose
        self.events=None

    def fit(self, coords, last_date=None):
        """
        :param coords: pd.Series
            Indexed and sorted by Date, with values = coords
        :param last_date: string (format='%Y-%m-%d') or DateTime, default None
            the last date of the time window. If None, the last date of coords is used
        """
        if self.tw is not None:
            if last_date is None:
                last_date = coords.index.max()
            elif isinstance(last_date, str):
                last_date = datetime.datetime.strptime(last_date, '%Y-%m-%d')
            # pandas time index slice include both begin and last date,
            # to have a time window=tw, the difference should be tw-1
            begin_date = last_date - datetime.timedelta(days=self.tw - 1)
            coords = coords.loc[begin_date:last_date]

        events = gp.GeoDataFrame(coords.apply(lambda x: Point(*x))).rename(columns={'coords': 'geometry'})
        self.events = events

In [3]:
tw = 60
bw = 400
verbose = 1

bower = Bower(bw=bw, tw=tw, verbose=verbose)

In [4]:
from src.bsln_kde import prep_data
from src.spatial_unit import baltimore_grids

In [5]:
grid_size = 200
grids = baltimore_grids(grid_size=grid_size)

In [6]:
data = prep_data('data/open-baltimore/raw/BPD_Part_1_Victim_Based_Crime_Data.csv', 
                 col_lon='Longitude', col_lat='Latitude', col_date='CrimeDate', to_epsg=3559)

In [7]:
coords = data.head(1000).coords

In [8]:
bower.fit(coords)

In [ ]:
bower.pred()

In [14]:
grids.iloc[[1,3,5]]

,geometry,cen_coords
1,"POLYGON ((438677.7622064584 169997.5488773746,...","(438577.76220645837, 170097.54887737456)"
3,"POLYGON ((439077.7622064584 169997.5488773746,...","(438977.76220645837, 170097.54887737456)"
5,"POLYGON ((438077.7622064584 170197.5488773746,...","(437977.76220645837, 170297.54887737456)"


In [46]:
from shapely.geometry import Point

In [49]:
data = grids.copy()
data['geometry'] = data.cen_coords.apply(lambda x: Point(*x))
data['geometry'] = data.buffer(bw)

In [52]:
events = gp.GeoDataFrame(coords.apply(lambda x: Point(*x))).rename(columns={'coords': 'geometry'}).reset_index()

In [53]:
joined = gp.sjoin(events, data)

In [90]:
joined.index_right.unique()

array([5416, 5418, 5417, ..., 2895, 2894, 2972], dtype=int64)

In [54]:
tmp = joined[joined.index_right==5415]

In [55]:
tmp

,Date,geometry,index_right,cen_coords
531,2013-01-05,POINT (434738.7188744294 187554.2880562786),5415,"(434577.76220645837, 187897.5488773746)"
683,2013-01-06,POINT (434868.8553430436 187751.372137545),5415,"(434577.76220645837, 187897.5488773746)"
962,2013-01-08,POINT (434868.8553430436 187751.372137545),5415,"(434577.76220645837, 187897.5488773746)"


In [24]:
import numpy as np

In [59]:
def get_distance(row):  

    a=np.array(row.geometry.coords[0])
    b=np.array(row.cen_coords)
    return np.linalg.norm(a-b)


In [86]:
def calc_risk(df):
    distance = df.apply(get_distance, axis=1)//(grid_size/2)+1
    n_weeks = df.Date.apply(lambda x: (now_date-x).days//7+1)
    return (1/distance*1/n_weeks).sum()

In [98]:
pred =joined[joined.index_right.isin([5416, 5418, 5417])].groupby('index_right').apply(calc_risk).reindex(grids.index).fillna(0)

In [99]:
pred

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
5961    0.0
5962    0.0
5963    0.0
5964    0.0
5965    0.0
5966    0.0
5967    0.0
5968    0.0
5969    0.0
5970    0.0
5971    0.0
5972    0.0
5973    0.0
5974    0.0
5975    0.0
5976    0.0
5977    0.0
5978    0.0
5979    0.0
5980    0.0
5981    0.0
5982    0.0
5983    0.0
5984    0.0
5985    0.0
5986    0.0
5987    0.0
5988    0.0
5989    0.0
5990    0.0
Length: 5991, dtype: float64

In [88]:
for i, group in joined.groupby('index_right'):
    print(group)
    break

         Date                                     geometry  index_right  \
49 2013-01-01  POINT (435169.9628933559 172807.2698065252)          241   

                                  cen_coords  
49  (434977.76220645837, 172497.54887737456)  


In [76]:
import datetime

In [77]:
now_date = datetime.datetime.strptime('2013-02-01', '%Y-%m-%d')

In [78]:
now_date - tmp.Date.iloc[0]

Timedelta('27 days 00:00:00')